<a href="https://colab.research.google.com/github/CiaraFarrellSETU/Dissertation_code/blob/main/resnet50_6classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from timeit import default_timer as timer

start = timer()

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/gdrive')

In [ ]:
import zipfile

with zipfile.ZipFile('/content/gdrive/MyDrive/dogeye.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzipp_d')

In [ ]:
base_dir = '/content/unzipp_d/dogeye'

train_dir = '/content/unzipp_d/dogeye/train'
cat_train_dir = '/content/unzipp_d/dogeye/train/cataract'
healthy_train_dir = '/content/unzipp_d/dogeye/train/healthy'
mas_train_dir = '/content/unzipp_d/dogeye/train/mas'
conj_train_dir = '/content/unzipp_d/dogeye/train/conj'
ulcer_train_dir = '/content/unzipp_d/dogeye/train/ulcer'
nuc_train_dir = '/content/unzipp_d/dogeye/train/nuc'

test_dir = '/content/unzipp_d/dogeye/test'
cat_test_dir = '/content/unzipp_d/dogeye/test/cataract'
healthy_test_dir = '/content/unzipp_d/dogeye/test/healthy'
mas_test_dir = '/content/unzipp_d/dogeye/test/mas'
conj_test_dir = '/content/unzipp_d/dogeye/test/conj'
ulcer_test_dir = '/content/unzipp_d/dogeye/test/ulcer'
nuc_test_dir = '/content/unzipp_d/dogeye/test/nuc'

val_dir = '/content/unzipp_d/dogeye/valid'
cat_val_dir = '/content/unzipp_d/dogeye/valid/cataract'
healthy_val_dir = '/content/unzipp_d/dogeye/valid/healthy'
mas_val_dir = '/content/unzipp_d/dogeye/valid/mas'
conj_val_dir = '/content/unzipp_d/dogeye/valid/conj'
ulcer_val_dir = '/content/unzipp_d/dogeye/valid/ucler'
nuc_val_dir = '/content/unzipp_d/dogeye/valid/nuc'

In [ ]:
import os
#train
num_cat_train = len(os.listdir(cat_train_dir))
num_healthy_train = len(os.listdir(healthy_train_dir))

num_mas_train = len(os.listdir(mas_train_dir))
num_conj_train = len(os.listdir(conj_train_dir))
num_ulcer_train = len(os.listdir(ulcer_train_dir))
num_nuc_train = len(os.listdir(nuc_train_dir))

#test
num_cat_test = len(os.listdir(cat_test_dir))
num_healthy_test = len(os.listdir(healthy_test_dir))

num_mas_test = len(os.listdir(mas_test_dir))
num_conj_test = len(os.listdir(conj_test_dir))
num_ulcer_test = len(os.listdir(ulcer_test_dir))
num_nuc_test = len(os.listdir(nuc_test_dir))

#val
num_cat_val = len(os.listdir(cat_val_dir))
num_healthy_val = len(os.listdir(healthy_val_dir))

num_mas_val = len(os.listdir(mas_val_dir))
num_conj_val = len(os.listdir(conj_val_dir))
num_ulcer_val = len(os.listdir(ulcer_val_dir))
num_nuc_val = len(os.listdir(nuc_val_dir))

In [ ]:
print("--")
print("Total Training cat Images",num_cat_train)
print("Total Training conj Images",num_conj_train)

print("Total Training healthy Images",num_healthy_train)
print("Total Training mas Images",num_mas_train)
print("Total Training nuc Images",num_nuc_train)
print("Total Training ulcer Images",num_ulcer_train)
print("--")
print("Total test cat Images",num_cat_test)
print("Total test healthy Images",num_healthy_test)

print("Total test mas Images",num_mas_test)
print("Total Training conj Images",num_conj_test)
print("Total Training ulcer Images",num_ulcer_test)
print("Total Training nuc Images",num_nuc_test)
print("--")
print("Total validation cat Images",num_cat_val)
print("Total validation healthy Images",num_healthy_val)

print("Total validation mas Images",num_mas_val)
print("Total Training conj Images",num_conj_val)
print("Total Training ulcer Images",num_ulcer_val)
print("Total Training nuc Images",num_nuc_val)
print("--")

In [ ]:
total_train = num_cat_train+num_mas_train+num_conj_train+num_ulcer_train+num_nuc_train+num_healthy_train
total_validation = num_cat_val+num_mas_val+num_conj_val+num_ulcer_val+num_nuc_val+num_healthy_val
total_test = num_cat_test+num_mas_test+num_conj_test+num_ulcer_test+num_nuc_test+num_healthy_test

total_images = total_train + total_validation + total_test

print("--")
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)
print("--")
print("Total Images",total_images)
print("--")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to train and test directories
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)  # Apply rescaling to test data too

# Load images from train and test directories with data augmentation for training
train_generator = train_datagen.flow_from_directory(
        '/content/unzipp_d/dogeye/train',
        target_size=(224, 224),  # Adjust image size as needed by your model
        batch_size=32,
        class_mode='categorical'  # Adjust based on your classification problem
)

# Load images from test directories (no augmentation for evaluation)
test_generator = test_datagen.flow_from_directory(
        '/content/unzipp_d/dogeye/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
        '/content/unzipp_d/dogeye/valid',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
)

In [ ]:
from tensorflow.keras.applications import ResNet50
# Load ResNet50, excluding the top (classification) layer
resnet = ResNet50(weights='imagenet', include_top=False)

# Freeze pre-trained layers to prevent retraining them
for layer in resnet.layers:
  layer.trainable = False

In [ ]:

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Add GlobalAveragePooling2D to reduce spatial dimensions of the output
x = resnet.output
x = GlobalAveragePooling2D()(x)

# Add a Dense layer with number of neurons equal to your number of classes
x = Dense(6, activation='softmax')(x)  ##Chnage back to 6

# Create the final model
model = Model(inputs=resnet.input, outputs=x)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)


In [ ]:
# model.fit(train_generator,
#           epochs=5,
#           validation_data=validation_generator)

In [ ]:
model.evaluate(test_generator)

In [ ]:

import numpy as np
y_pred = model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)
y_true = test_generator.classes
print(classification_report(y_true, y_pred))


In [ ]:
import sklearn.metrics as metrics

In [ ]:

confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
print(confusion_matrix)

In [ ]:
end = timer()
print(end - start)